In [ ]:
import random
import os
import shutil
import copy
from skimage import measure
from skimage import morphology as skim
import skimage.io as skio
import warnings
import pandas as pd
import numpy as np
import pathlib
import geopandas as gpd
from rasterio import features, coords
from rasterio.plot import reshape_as_raster
import rasterio
from shapely.geometry import shape
from osgeo import gdal
from itertools import product
from rasterio import windows
from multiprocessing.pool import ThreadPool
from cropmask.label_prep import rio_bbox_to_polygon
from cropmask.misc import parse_yaml, make_dirs
from cropmask import grid

In [ ]:
import cropmask.preprocess as pp
from cropmask.misc import make_dirs, remove_dirs
wflow = pp.PreprocessWorkflow("/home/ryan/work/CropMask_RCNN/cropmask/preprocess_config.yaml", 
                                 "/mnt/point/western_nebraska_landsat_scenes/LT050320312005082801T1-SC20190418222350/",
                                 "/mnt/point/external/nebraska_pivots_projected.geojson")

Need to see why the remove function causes label not found error 

In [ ]:
directory_list = wflow.setup_dirs()

band_list = wflow.yaml_to_band_index()

product_list = wflow.get_product_paths(band_list)

stacked_arr = wflow.load_and_stack_bands(product_list)

wflow.stack_and_save_bands()

wflow.negative_buffer_and_small_filter(-31, 100)

paths_tuple = wflow.grid_images()

In [ ]:
paths_tuple

In [ ]:
help(os.path.split)

In [ ]:
wflow.move_chips_to_folder()

wflow.connected_components()

In [ ]:
gridded_label_ids = [os.path.splitext(i)[0] for i in os.listdir(wflow.GRIDDED_LABELS)]

In [ ]:
gridded_img_ids = [os.path.splitext(i)[0] for i in os.listdir(wflow.GRIDDED_IMGS)]

In [ ]:
len(os.listdir(wflow.TRAIN))==len(os.listdir(wflow.GRIDDED_LABELS))

In [ ]:
gridded_ids = [os.path.splitext(i)[0][:-6] for i in os.listdir(wflow.GRIDDED_LABELS)]

In [ ]:
train_ids = os.listdir(wflow.TRAIN)

In [ ]:
set(gridded_ids)-set(train_ids)

In [ ]:
chip_img_paths = grid.grid_images_rasterio_controlled_threads(wflow.rasterized_label_path, wflow.GRIDDED_LABELS, output_name_template='tile_{}-{}.tif', grid_size=wflow.grid_size)

In [ ]:
chip_img_paths

In [ ]:
grid_result = [os.path.splitext(os.path.basename(i)) for i in chip_img_paths]

In [ ]:
for chip_id in os.listdir(self.GRIDDED_IMGS):
    chip_id = os.path.splitext(chip_id)[0]

In [ ]:
for chip_id in os.listdir(wflow.TRAIN):
    chip_label_path = os.path.join(wflow.GRIDDED_LABELS, chip_id) + "_label.tif"
    print(chip_label_path)

In [ ]:
    arr = skio.imread(chip_label_path)
    blob_labels = measure.label(arr, background=0)
    blob_vals = np.unique(blob_labels)
    # for imgs with no instances, create empty mask
    if len(blob_vals) == 1:
        img_chip_folder = os.path.join(wflow.TRAIN, chip_id)
        img_chip_name = os.listdir(img_chip_folder)[0]
        img_chip_path = os.path.join(img_chip_folder, img_chip_name)
        arr = skio.imread(img_chip_path)
        mask = np.zeros_like(arr[:, :, 0])
        mask_folder = os.path.join(wflow.TRAIN, chip_id, "mask")
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            skio.imsave(os.path.join(mask_folder, chip_id + "_0.tif"), mask)
    else:
        # only run connected comp if there is at least one instance
        for blob_val in blob_vals[blob_vals != 0]:
            labels_copy = blob_labels.copy()
            labels_copy[blob_labels != blob_val] = 0
            labels_copy[blob_labels == blob_val] = 1
            label_name = chip_id + "_label_" + str(blob_val) + ".tif"
            mask_path = os.path.join(wflow.TRAIN, chip_id, "mask")
            label_path = os.path.join(mask_path, label_name)
            assert labels_copy.ndim == 2
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=UserWarning)
                skio.imsave(label_path, labels_copy)

In [ ]:
arr = skio.imread('/mnt/point/landsat-1024-cp/train/tile_1024-5120/image/tile_1024-5120_.tif')

In [ ]:
%matplotlib inline
skio.imshow(arr[:,:,0])

In [ ]:
import os
from itertools import product
import rasterio as rio
from rasterio import windows
import threading

from multiprocessing.dummy import Pool as ThreadPool

In [ ]:
def get_tiles_for_threaded_map(ds, width=256, height=256):
    """
        Returns a list of tuple where each tuple is the window and transform information for the image chip. 
    """
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    chip_list = []
    def get_win(ds, col_off, row_off, width, height, big_window):
        """
        Returns the bounds of each image chip/tile as a rasterio window object as well as the transform
        as a tuple like (rasterio.windows.Window, transform)
        """
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        return (window, transform)
    chip_list = list(map(lambda x: get_win(ds, x[0], x[1], width, height, big_window), offsets))
    return chip_list

def write_by_window(ds, out_dir, output_name_template, meta, window, transform):
    """Writes an image chip given the source meta, the chip window, and the chip transform"""
    meta['transform'] = transform
    meta['width'], meta['height'] = window.width, window.height
    outpath = os.path.join(out_dir,output_name_template.format(int(window.col_off), int(window.row_off)))
    with rasterio.open(outpath, 'w', **meta) as outds:
        outds.write(ds.read(window=window))
    return outpath

def map_threads(func, sequence, threads=4):
    pool = ThreadPool(threads)
    results = pool.map(func, sequence)
    pool.close()
    pool.join()
    return results

def grid_images_rasterio_controlled_threads(in_path, out_dir, output_name_template='tile_{}-{}.tif'):
    """
    Reads in a Landsat scene and writes out square image chips using a thread pool.
    """    
    with rasterio.open(in_path) as src:
        meta = src.meta.copy()
        chip_list = get_tiles_for_threaded_map(src)
        out_paths = list(map_threads(lambda x: write_by_window(src, out_dir, output_name_template, meta, x[0], x[1]), chip_list))
    return out_paths

def grid_images_rasterio_regular_map(in_path, out_dir, output_name_template='tile_{}-{}.tif'):
    """
    Grids up imagery to a variable size. Filters out imagery with too little usable data.
    appends a random unique id to each tif and label pair, appending string 'label' to the 
    mask. Inspired by https://gis.stackexchange.com/questions/285499/how-to-split-multiband-image-into-image-tiles-using-rasterio
    """    
    with rasterio.open(in_path) as src:
        meta = src.meta.copy()
        chip_list = get_tiles_for_threaded_map(src)
        out_paths = list(map(lambda x: write_by_window(src, out_dir, output_name_template, meta, x[0], x[1]), chip_list))
    return out_paths

In [ ]:
%%time
with rasterio.open(in_path) as inds:
    chip_list = get_tiles_for_threaded_map(inds)

In [ ]:
%%time
out_paths = grid_images_rasterio_controlled_threads(wflow.stacked_path, wflow.GRIDDED_IMGS)

In [ ]:
%%time
out_paths = grid_images_rasterio_regular_map(wflow.stacked_path, wflow.GRIDDED_IMGS)